In [1]:
from sqlalchemy import URL, create_engine, text
import streamlit as st

In [2]:
connection_string = URL.create(
    'postgresql',
    username=st.secrets.connections.postgresql.username,
    password=st.secrets.connections.postgresql.password,
    host=st.secrets.connections.postgresql.host,
    database=st.secrets.connections.postgresql.database,
    query={'options':'endpoint=ep-sweet-cloud-a1zh9huf'}
)

In [3]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'}, pool_pre_ping=True, pool_recycle=3600)
conn = engine.connect()

## Delete

In [4]:
query = text('SELECT url , COUNT(url) FROM news GROUP BY url HAVING COUNT(url) > 1')
output = conn.execute(query)
rows = output.fetchall()
for row in rows:
    print(row)

('https://e.vnexpress.net/news/travel/food-recipes/vietnamese-cuisine-popularized-in-singapore-4747133.html', 3)
('https://e.vnexpress.net/news/news/s-korea-employment-applications-from-vietnam-exceed-quota-threefold-4747066.html', 2)
('https://e.vnexpress.net/news/news/buddha-statues-unearthed-in-laos-s-bokeo-province-4747807.html', 2)
('https://e.vnexpress.net/news/travel/three-foreign-tourists-shot-dead-in-central-afghanistan-4747554.html', 3)
('https://e.vnexpress.net/news/travel/singapore-s-changi-airport-allows-all-foreigners-to-use-automated-lanes-4747674.html', 3)
('https://e.vnexpress.net/news/news/traffic/four-face-charges-for-parking-luxury-cars-on-road-for-wedding-photo-shoot-4747868.html', 2)
('https://e.vnexpress.net/news/sports/other-sports/vietnamese-player-to-miss-american-basketball-dream-4747376.html', 2)
('https://e.vnexpress.net/news/news/environment/saigon-canal-cleaned-of-rubbish-blocking-water-flow-4747269.html', 3)
('https://e.vnexpress.net/news/travel/china-al

In [5]:
query = text('DELETE FROM news a USING news b WHERE a.id < b.id AND a.url = b.url')
conn.execute(query)

In [6]:
query = text('SELECT url , COUNT(url) FROM news GROUP BY url HAVING COUNT(url) > 1')
output = conn.execute(query)
rows = output.fetchall()
for row in rows:
    print(row)

In [7]:
conn.commit()

In [8]:
conn.close()

## Crawl and insert

In [4]:
from _crawler import FilterURL, get_all_links, get_content
from _summarizer import create_model, summarize

summarizer_model = create_model('gemini-1.0-pro')

In [12]:
news_sample = get_content('https://e.vnexpress.net/news/news/singapore-allows-online-declaration-for-15-000-cash-transit-4736850.html')
summarized_text_sample = summarize(summarizer_model, full_text=news_sample['text'])

In [13]:
summarized_text_sample = summarized_text_sample.replace("'", "''").replace('"', '')
news_sample['title'] = news_sample['title'].replace('"', '')

In [14]:
query_insert = "INSERT INTO news(url, title, date, author, category, summary) " + \
                f"VALUES('{news_sample['url']}', '{news_sample['title']}', '{news_sample['date']}', '{news_sample['author']}', '{news_sample['category']}', '{summarized_text_sample}')"

In [15]:
query_insert_check = "INSERT INTO NEWS(url, title, date, author, category, summary) " + \
                f"SELECT '{news_sample['url']}', '{news_sample['title']}', '{news_sample['date']}', '{news_sample['author']}', '{news_sample['category']}', '{summarized_text_sample}'" + \
                f"WHERE NOT EXISTS (SELECT 1 FROM NEWS WHERE url = '{news_sample['url']}')"

In [16]:
query_insert_check

"INSERT INTO NEWS(url, title, date, author, category, summary) SELECT 'https://e.vnexpress.net/news/news/singapore-allows-online-declaration-for-15-000-cash-transit-4736850.html', 'Singapore allows online declaration for $15,000 cash transit', '2024-04-21', 'Hoang Phong', 'news', '**Online Cash Declaration for Singapore Travelers:**\n\nEffective May 13, travelers entering or leaving Singapore with cash exceeding S$20,000 must make online declarations.\n\nThe online form, available from May 10 on the Immigration and Checkpoints Authority website or MyICA app, aims to streamline the process and enhance convenience.\n\nDeclarations must be submitted three days prior to arrival/departure and reviewed before reaching Singapore checkpoints.\n\nFailure to declare cash and negotiable instruments can result in imprisonment up to three years, fines up to S$50,000, or both.'WHERE NOT EXISTS (SELECT 1 FROM NEWS WHERE url = 'https://e.vnexpress.net/news/news/singapore-allows-online-declaration-for-

In [17]:
conn.execute(text(query_insert_check))

In [20]:
conn.commit()

In [21]:
output = conn.execute(text("SELECT * FROM news"))

In [22]:
rows = output.fetchall()
for row in rows:
    print(row)

(1, 'https://e.vnexpress.net/news/property/asking-prices-for-uk-homes-close-to-record-high-4737334.html', 'Asking prices for UK homes close to record high', datetime.date(2024, 4, 22), 'Reuters', 'property', '**Summary:**\\n\\n* UK house asking prices are near record highs, increasing by 1.7% annually.\\n* Monthly asking prices rose by 1.1%, slowing from t ... (290 characters truncated) ... hile growth is slower among first-time and second-time buyers.\\n* Experts caution against substantial price growth due to current market conditions.')
(3, 'https://e.vnexpress.net/news/news/education/hcmc-private-university-annual-tuition-cost-nearly-14-000-4734709.html', 'HCMC private university annual tuition cost nearly $14,000', datetime.date(2024, 4, 18), 'Duong Tam', 'news', '**Tuition Fees for Private Universities in Ho Chi Minh City**\n\nAnnual tuition fees for new students at private universities in Ho Chi Minh City (HC ... (599 characters truncated) ... onsidering proficiency tests and En

In [23]:
conn.commit()

In [29]:
conn.close()